Import packages and activate MATLAB engine for calling the SPHARM decomposition function 

In [ ]:
from os.path import join,basename,isdir
from os import mkdir
from glob import glob
import matlab.engine
import numpy as np
import scipy.io as sio
import pandas as pd

eng = matlab.engine.start_matlab()

eng.addpath('functionDep')

# path to the NFSFT matlab library included in the following package
# https://www-user.tu-chemnitz.de/~potts/nfft/
# please change to correct path on your system
nfsft_path = '/path/to/nfsft/'
eng.addpath(nfsft_path)


Provide paths to segmentations and images and list files in directories

In [25]:
mypathseg   = 'Path to segmentations'
mypathimg   = 'Path to images'

mypathsave  = 'Path to where SPHARM matrices will be saved'

if(not isdir(mypathsave)):
    mkdir(mypathsave)

# List of input image files 
# Filenames are assumed to be the same within the segmentation and image directories
# Images are expected to be in nifti format (.nii or .nii.gz)
input_images = []
for file in glob(join(mypathimg,'*.nii*')):
    input_images.append(basename(file))

input_images.sort()


In [26]:
#This is the path to a CSV file with two columns, assumed to be named "ID" and "Label"
# see README file for more information

mypath_labelsCSV = '/path/to/csv/file/with/binary/labels.csv'

outcomes_df=pd.read_csv(mypath_labelsCSV)
outcomes_df.set_index('ID',inplace=True)

Apply SPHARM decomposition of segmented regions in images

In [ ]:
R_max = 25.0 #needs to be float, not int
L_max = 25.0 #needs to be float, not int

for i in range(len(input_images)):
    current_img = input_images[i]
    img_ID = current_img.split('.')
    img_ID = img_ID[0]
    
    # Calculate SPHARM decomposition matrix
    flmr_in = eng.fun_spharm(join(mypathimg,input_images[i]),join(mypathseg,input_images[i]),R_max,L_max)
    flmr_in = np.asarray(flmr_in)

    label = outcomes_df.loc[img_ID, 'Label']

    # Save SPHARM decomposition matrix and its associated label as a Matlab file
    sio.savemat((join(mypathsave,img_ID)+'.mat'),{'flmr_in':flmr_in, 'label':label})
    print('Exported '+img_ID)